In [31]:
from pathlib import Path

import pandas as pd
from collections import defaultdict

pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

In [32]:
dir = "/Volumes/USB SanDisk 3.2Gen1 Media 1/a4d/a4dphase2_upload/output/logs"

In [121]:
files = list(Path(dir).glob("*.log"))
len(files)

78

In [122]:
data = defaultdict(list)
for file in files:
    with open(file, "r", encoding="utf-8") as fstream:
        for line in fstream.read().splitlines():
            error = None
            warn = None
            if "ERROR" in line:
                error = line.split("\t")[-1]
                
            if "WARN" in line:
                warn = line.split("\t")[-1]
            
            if error or warn: 
                data["file"].append(file.name)
                data["error"].append(error)
                data["warning"].append(warn)

In [123]:
df = pd.DataFrame(data)

In [124]:
df

,file,error,warning
0,2020_Penang General Hospital A4D Tracker_DC_patient_raw.log,None,Found invalid values for column blood_pressure_mmhg that do not follow the format X/Y. Values were replaced with 999999.
1,2020_Penang General Hospital A4D Tracker_DC_patient_raw.log,None,"Extra columns in patient data: admissiontohospitaldatemmmyy, observations1, admissiontohospitaldatemmyy"
2,2020_Penang General Hospital A4D Tracker_DC_patient_raw.log,None,"Missing columns in patient data: clinic_code, country_code, fbg_baseline_mg, fbg_updated_mg, hospitalisation_cause, hospitalisation_date, last_remote_followup_date, lost_date, observations, observations_category, status_out, t1d_diagnosis_date, t1d_diagnosis_with_dka, updated_2022_date"
3,2020_Penang General Hospital A4D Tracker_DC_patient_raw.log,None,Could not convert value - in column bmi for patient: MY_PN004
4,2020_Penang General Hospital A4D Tracker_DC_patient_raw.log,None,Could not convert value - in column bmi for patient: MY_PN004
...,...,...,...
9971,2022_CDA A4D Tracker_patient_raw.log,None,"Patient KH_CD015: Value Premixed 30/70 BD,Basal-bolus (AN) for column insulin_regimen is not in the list of allowed values."
9972,2022_CDA A4D Tracker_patient_raw.log,None,"Patient KH_CD015: Value Premixed 30/70 BD,Basal-bolus (AN) for column insulin_regimen is not in the list of allowed values."
9973,2022_CDA A4D Tracker_patient_raw.log,None,"Patient KH_CD015: Value Premixed 30/70 BD,Basal-bolus (AN) for column insulin_regimen is not in the list of allowed values."
9974,2022_CDA A4D Tracker_patient_raw.log,None,"Patient KH_CD015: Value Premixed 30/70 BD,Basal-bolus (AN) for column insulin_regimen is not in the list of allowed values."


In [125]:
df.to_csv(Path(dir) / "error_report.csv", index=False)

In [126]:
df.file.nunique()

58

In [127]:
df.file.unique()

array(['2020_Penang General Hospital A4D Tracker_DC_patient_raw.log',
       '2020_Mahosot Hospital A4D Tracker_DC_product_raw.log',
       '2019_Sarawak General Hospital A4D Tracker_DC_patient_raw.log',
       '2022_Sultanah Bahiyah A4D Tracker_DC_patient_raw.log',
       '2022_Khon Kaen Hospital A4D Tracker_patient_raw.log',
       '2020_Lao Friends Hospital for Children A4D Tracker_DC_product_raw.log',
       '2019_Lao Friends Hospital for Children A4D Tracker_DC_patient_raw.log',
       '2022_Sarawak General Hospital A4D Tracker_DC_product_raw.log',
       '2021_Mahosot Hospital A4D Tracker_DC_product_raw.log',
       '2022_Surat Thani A4D Tracker_product_raw.log',
       '2022_Chulalongkorn Hospital A4D Tracker_patient_raw.log',
       '2019_Penang General Hospital A4D Tracker_DC_patient_raw.log',
       '2019_Sultanah Bahiyah Hospital A4D Tracker_DC_patient_raw.log',
       '2020_Sarawak General Hospital A4D Tracker_DC_product_raw.log',
       '2019_Mahosot Hospital A4D Tracker_p

## Error

In [128]:
df[~df.error.isna()]

,file,error,warning
698,2020_Sarawak General Hospital A4D Tracker_DC_product_raw.log,Could not process raw product data. Error = Error in `dplyr::mutate()`: ℹ In argument: `product_received_from = case_when(...)`. Caused by error in `case_when()`: ! Can't combine `..1 (right)` <double> and `..2 (right)` <character>. .,None
9734,2017_Mahosot Hospital A4D Tracker_product_raw.log,Could not process raw product data. Error = Error in `[<-` at R/helper_product_data.R:483:12: ! Assigned data `value` must be compatible with existing data. ℹ Error occurred for column `product_units_released`. Caused by error in `vec_assign()`: ! Can't convert <double> to <character>. .,None


In [129]:
# how many files could not be processed in %?
len(df[~df.error.isna()]) / len(files) * 100
                                                        

2.564102564102564

### Error analysis

#### Missing or invalid patient id

In [130]:
# missing patient id
df[(~df.error.isna()) & (df.error.str.contains("id is not valid"))].drop_duplicates()

,file,error,warning


#### missing month

In [131]:
# month list is empty?
print("\n".join(l.split(".")[0] for l in df[(~df.error.isna()) & (df.error.str.contains("month_list"))].file.tolist()))

#### missing patient data

In [132]:
# cannot find patient data
for year in range(2017,2023):
    subdf = df[(~df.error.isna()) & (df.error.str.contains("readxl::cell_limits"))]
    print(subdf[subdf.file.str.startswith(str(year))].file.tolist())

[]
[]
[]
[]
[]
[]


In [133]:
# check problem with num_na_rows
subdf = df[(~df.error.isna()) & (df.error.str.contains("num_na_rows"))]
subdf

,file,error,warning


In [134]:
# check start_df_msd:end_df_msd
subdf = df[(~df.error.isna()) & (df.error.str.contains("start_df_msd"))]
subdf.file.tolist()

[]

# Warnings

In [135]:
df[~df.warning.isna()]

,file,error,warning
0,2020_Penang General Hospital A4D Tracker_DC_patient_raw.log,None,Found invalid values for column blood_pressure_mmhg that do not follow the format X/Y. Values were replaced with 999999.
1,2020_Penang General Hospital A4D Tracker_DC_patient_raw.log,None,"Extra columns in patient data: admissiontohospitaldatemmmyy, observations1, admissiontohospitaldatemmyy"
2,2020_Penang General Hospital A4D Tracker_DC_patient_raw.log,None,"Missing columns in patient data: clinic_code, country_code, fbg_baseline_mg, fbg_updated_mg, hospitalisation_cause, hospitalisation_date, last_remote_followup_date, lost_date, observations, observations_category, status_out, t1d_diagnosis_date, t1d_diagnosis_with_dka, updated_2022_date"
3,2020_Penang General Hospital A4D Tracker_DC_patient_raw.log,None,Could not convert value - in column bmi for patient: MY_PN004
4,2020_Penang General Hospital A4D Tracker_DC_patient_raw.log,None,Could not convert value - in column bmi for patient: MY_PN004
...,...,...,...
9971,2022_CDA A4D Tracker_patient_raw.log,None,"Patient KH_CD015: Value Premixed 30/70 BD,Basal-bolus (AN) for column insulin_regimen is not in the list of allowed values."
9972,2022_CDA A4D Tracker_patient_raw.log,None,"Patient KH_CD015: Value Premixed 30/70 BD,Basal-bolus (AN) for column insulin_regimen is not in the list of allowed values."
9973,2022_CDA A4D Tracker_patient_raw.log,None,"Patient KH_CD015: Value Premixed 30/70 BD,Basal-bolus (AN) for column insulin_regimen is not in the list of allowed values."
9974,2022_CDA A4D Tracker_patient_raw.log,None,"Patient KH_CD015: Value Premixed 30/70 BD,Basal-bolus (AN) for column insulin_regimen is not in the list of allowed values."


### Warning analysis

#### extra columns

In [154]:
missed_names = df[(~df.warning.isna()) & (df.warning.str.contains("Extra"))].warning.str.strip("Extra columns in patient data:").drop_duplicates().to_list()

names = set()
for x in missed_names:
    names.update(x for x in x.split(",") if x)
    
names

{' admissiontohospitaldatemmmyy',
 ' admissiontohospitaldatemmyy',
 ' complication_screening',
 ' complication_screening_',
 ' complication_screening_date',
 ' complication_screening_results',
 ' est_strips_pmoth',
 ' family_support_',
 ' family_support_scale',
 ' hospitalisation_date1',
 ' instantmeterreceiv',
 ' instantmeterreceiveddate',
 ' insulin_required_month',
 ' insulin_required_y',
 ' lastclinicvisitไปโรงพยาบาลdate',
 ' lastremotefollowupสายเขา',
 ' lastremotefollowupโดยโทรศัพทdate',
 ' meter_received_',
 ' meter_received_date',
 ' new',
 ' observations1',
 ' remote_followup',
 ' remotefu',
 ' virtualremotecontactdate',
 ' virtualremotecontactyn',
 '_dosage',
 '_screening',
 '_screening_results',
 '_strips_pmoth',
 '_visit',
 'hospitaldatemmmyy'}

In [ ]:
names = set()
for row in missed_names:
    names.update(w.strip(".") for w in row.split(","))

In [46]:
names

NameError: name 'names' is not defined

#### parsing dates

In [61]:
subdf = df[(~df.warning.isna()) & (df.warning.str.contains("Could not convert value", regex=False))]

subdf.warning.str.strip("Could not convert value ").unique().tolist()

['- in column bmi for patient: MY_PN004',
 '- in column fbg_baseline_mmol for patient: MY_PN004',
 'Sept-20 in column bmi_date for patient: MY_PN007',
 'Sept-20 in column bmi_date for patient: MY_PN008',
 'sept-20 in column bmi_date for patient: MY_PN010',
 '0ct/19 in column bmi_date for patient: MY_PN011',
 'il in column hba1c_baseline for patient: MY_SW008',
 'NA or Hospitalisation Date in column hospitalisation_date for patient: MY_SW001',
 'NA or Hospitalisation Date in column hospitalisation_date for patient: MY_SW002',
 'NA or Hospitalisation Date in column hospitalisation_date for patient: MY_SW003',
 'NA or Hospitalisation Date in column hospitalisation_date for patient: MY_SW004',
 'NA or Hospitalisation Date in column hospitalisation_date for patient: MY_SW005',
 'NA or Hospitalisation Date in column hospitalisation_date for patient: MY_SW009',
 'NA or Hospitalisation Date in column hospitalisation_date for patient: MY_SW010',
 'NA or Hospitalisation Date in column hospitalis

In [55]:
subdf.file.unique().tolist()

['2020_Penang General Hospital A4D Tracker_DC_patient_raw.log',
 '2019_Sarawak General Hospital A4D Tracker_DC_patient_raw.log',
 '2022_Sultanah Bahiyah A4D Tracker_DC_patient_raw.log',
 '2022_Khon Kaen Hospital A4D Tracker_patient_raw.log',
 '2019_Lao Friends Hospital for Children A4D Tracker_DC_patient_raw.log',
 '2019_Penang General Hospital A4D Tracker_DC_patient_raw.log',
 '2019_Sultanah Bahiyah Hospital A4D Tracker_DC_patient_raw.log',
 '2019_Mahosot Hospital A4D Tracker_patient_raw.log',
 '2021_Penang General Hospital A4D Tracker_DC_patient_raw.log',
 '2020_Lao Friends Hospital for Children A4D Tracker_DC_patient_raw.log',
 '2022_Sarawak General Hospital A4D Tracker_DC_patient_raw.log',
 '2021_Mahosot Hospital A4D Tracker_DC_patient_raw.log',
 '2018_Penang General Hospital A4D Tracker_DC_patient_raw.log',
 '2020_Sarawak General Hospital A4D Tracker_DC_patient_raw.log',
 '2021_NPH A4D Tracker_patient_raw.log',
 '2022_Sunprasitthiprasong Hospital A4D Tracker_patient_raw.log',
 "20

#### invalid values (outside range)

In [66]:
subdf = df[(~df.warning.isna()) & (df.warning.str.contains("Found invalid value", regex=False))]

subdf.warning.str.strip("Found invalid value ").str.split(expand=True)[[0,3]].drop_duplicates()

,0,3
0,s,blood_pressure_mmhg
143,18.5,hba1c_baseline
147,200,fbg_baseline_mmol
220,122,age
232,0,hba1c_baseline
773,11,blood_pressure_sys_mmhg
777,12,blood_pressure_sys_mmhg
790,15,blood_pressure_sys_mmhg
794,17,blood_pressure_sys_mmhg
797,19,blood_pressure_sys_mmhg


In [68]:
subdf.file.unique().tolist()

['2022_Sultanah Bahiyah A4D Tracker_DC_patient_raw.log',
 '2022_Khon Kaen Hospital A4D Tracker_patient_raw.log',
 '2022_Chulalongkorn Hospital A4D Tracker_patient_raw.log',
 '2021_Penang General Hospital A4D Tracker_DC_patient_raw.log',
 '2020_Lao Friends Hospital for Children A4D Tracker_DC_patient_raw.log',
 '2022_Putrajaya Hospital A4D Tracker_DC_patient_raw.log',
 '2022_Surat Thani A4D Tracker_patient_raw.log',
 '2022_Sarawak General Hospital A4D Tracker_DC_patient_raw.log',
 '2021_Mahosot Hospital A4D Tracker_DC_patient_raw.log',
 '2018_Penang General Hospital A4D Tracker_DC_patient_raw.log',
 '2020_Sarawak General Hospital A4D Tracker_DC_patient_raw.log',
 '2021_NPH A4D Tracker_patient_raw.log',
 '2022_Sunprasitthiprasong Hospital A4D Tracker_patient_raw.log',
 '2022_Phattalung Hospital A4D Tracker_patient_raw.log',
 "2021_Likas Women & Children's Hospital A4D Tracker_DC_patient_raw.log",
 '2021_Lao Friends Hospital for Children A4D Tracker_DC_patient_raw.log',
 '2022_Pahol  Polp

#### invalid values (not allowed)

In [76]:
subdf = df[(~df.warning.isna()) & (df.warning.str.contains("not in the list of allowed values", regex=False))]

subdf.warning.str.split(expand=True)[[3, 6]].drop_duplicates()

,3,6
244,"NA,NA",insulin_regimen
256,"Basal-bolus,NA",insulin_regimen
265,Basal-bolus,for
381,Modified,conventional
393,Self-mixed,for
417,"Basal-bolus,Basal-bolus",insulin_regimen
501,Self-mixed,column
683,"Other,NA",insulin_regimen
1207,Basal-bolos,insulin_regimen
1214,Others,insulin_regimen


In [75]:
subdf.file.unique().tolist()

['2022_Sultanah Bahiyah A4D Tracker_DC_patient_raw.log',
 '2022_Khon Kaen Hospital A4D Tracker_patient_raw.log',
 '2022_Chulalongkorn Hospital A4D Tracker_patient_raw.log',
 '2021_Penang General Hospital A4D Tracker_DC_patient_raw.log',
 '2020_Lao Friends Hospital for Children A4D Tracker_DC_patient_raw.log',
 '2022_Putrajaya Hospital A4D Tracker_DC_patient_raw.log',
 '2022_Surat Thani A4D Tracker_patient_raw.log',
 '2022_Sarawak General Hospital A4D Tracker_DC_patient_raw.log',
 '2021_Mahosot Hospital A4D Tracker_DC_patient_raw.log',
 '2018_Penang General Hospital A4D Tracker_DC_patient_raw.log',
 '2020_Sarawak General Hospital A4D Tracker_DC_patient_raw.log',
 '2021_NPH A4D Tracker_patient_raw.log',
 '2022_Sunprasitthiprasong Hospital A4D Tracker_patient_raw.log',
 '2022_Phattalung Hospital A4D Tracker_patient_raw.log',
 "2021_Likas Women & Children's Hospital A4D Tracker_DC_patient_raw.log",
 '2021_Lao Friends Hospital for Children A4D Tracker_DC_patient_raw.log',
 '2022_Pahol  Polp